# Multi-Class Prediction of Obesity Risk


## Preparação do ambiente e dos dados


### Imports e Configurações Gerais


In [188]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
# import numpy as np
# import missingno as msno
# import json
# import math

from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.pipeline import Pipeline


from sklearn.preprocessing import LabelEncoder

from sklearn.ensemble import RandomForestClassifier

%matplotlib inline

# seaborn.set_theme()

### Funções


#### invalid_values_count ( )


In [189]:
def show_stats(dataframe):
    """
    Retorna dataframe contendo a contagem de valores inválidos (None, NaN, NaT) em cada coluna do dataframe passado como parâmetro.
    """

    data_dict = {
        c: [dataframe.columns[c], (dataframe.shape[0] - dataframe[dataframe.columns[c]].count()), len(dataframe[dataframe.columns[c]].unique()), dataframe[dataframe.columns[c]].dtype]
        for c in range(len(dataframe.columns))
    }

    return pd.DataFrame(data=data_dict, index=['Nome', 'Val. inválidos', 'Val. únicos', 'Tipo']).T

### Carregamento dos Dados Rotulados (Completos)


In [190]:
train = pd.read_csv('../data/train.csv')
test = pd.read_csv('../data/test.csv')

## Análise Exploratória


### Visão Geral


Os dados estão todos completos, sem valores faltantes ou evidentemente inválidos (ex: NaN) em nenhuma coluna.

Há [Informações sobre o significado de cada coluna](https://www.kaggle.com/competitions/playground-series-s4e2/discussion/472516) em comentário no Kaggle.


In [191]:
train.head()

,id,Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,MTRANS,NObeyesdad
0,0,Male,24.443011,1.699998,81.669950,yes,yes,2.000000,2.983297,Sometimes,no,2.763573,no,0.000000,0.976473,Sometimes,Public_Transportation,Overweight_Level_II
1,1,Female,18.000000,1.560000,57.000000,yes,yes,2.000000,3.000000,Frequently,no,2.000000,no,1.000000,1.000000,no,Automobile,Normal_Weight
2,2,Female,18.000000,1.711460,50.165754,yes,yes,1.880534,1.411685,Sometimes,no,1.910378,no,0.866045,1.673584,no,Public_Transportation,Insufficient_Weight
3,3,Female,20.952737,1.710730,131.274851,yes,yes,3.000000,3.000000,Sometimes,no,1.674061,no,1.467863,0.780199,Sometimes,Public_Transportation,Obesity_Type_III
4,4,Male,31.641081,1.914186,93.798055,yes,yes,2.679664,1.971472,Sometimes,no,1.979848,no,1.967973,0.931721,Sometimes,Public_Transportation,Overweight_Level_II


In [192]:
show_stats(train)

,Nome,Val. inválidos,Val. únicos,Tipo
0,id,0,20758,int64
1,Gender,0,2,object
2,Age,0,1703,float64
3,Height,0,1833,float64
4,Weight,0,1979,float64
5,family_history_with_overweight,0,2,object
6,FAVC,0,2,object
7,FCVC,0,934,float64
8,NCP,0,689,float64
9,CAEC,0,4,object


In [193]:
binary_columns = ['Gender', 'family_history_with_overweight', 'FAVC', 'SMOKE', 'SCC']
categorical_columns = ['CAEC', 'CALC', 'MTRANS', 'NObeyesdad']
numerical_columns = ['Age', 'Height', 'Weight', 'FCVC', 'NCP', 'CH2O', 'FAF', 'TUE']

### Atributos categóricos


#### Gender

Pela característica do atributo (não ter uma ordem), certamente vamos utilizar o One Hot Encoding. Que neste caso será como um atributo binário porque por conta da opção drop="First" que será usada.

In [194]:
print(train.Gender.unique())
print(test.Gender.unique())

['Male' 'Female']
['Male' 'Female']


In [195]:
gender_options = ['Male', 'Female']

#### CALC


Para este atributo há uma ordem muito clara entre as classes, de maneira que podemos fazer ordinal encoding, mas também podemos fazer one hot encoding, que possui a vantagem de não atribuir uma escala às classes. Mas ambos os métodos podem ser testados.


In [196]:
print(train.CALC.unique())
print(test.CALC.unique())

['Sometimes' 'no' 'Frequently']
['Sometimes' 'no' 'Frequently' 'Always']


In [197]:
CALC_options = ['no', 'Sometimes', 'Frequently', 'Always']

#### MTRANS


Neste atributo podemos proceder de duas maneiras principais:

- Utilizar One Hot Encoding. Escolha que considero mais certeira por não atribuir ordem ou escala.
- Definir uma ordem e utilizar ordinal encoding. Neste caso podemos olhar o comportamento do atributo em relação ao target para validar a ordem escolhida para cada classe.


In [198]:
print(train.MTRANS.unique())
print(test.MTRANS.unique())

['Public_Transportation' 'Automobile' 'Walking' 'Motorbike' 'Bike']
['Public_Transportation' 'Automobile' 'Walking' 'Bike' 'Motorbike']


In [199]:
MTRANS_options = ['Walking', 'Public_Transportation', 'Bike', 'Motorbike', 'Automobile']

#### NObeyesdad (Target)


Para este atributo será feito ordinal encoding, porque há uma ordem clara entre as classes e este é o target, portanto está é a técnica mais usual para que a saída do algoritmo possa ser numérica.


In [200]:
print(train.NObeyesdad.unique())

['Overweight_Level_II' 'Normal_Weight' 'Insufficient_Weight'
 'Obesity_Type_III' 'Obesity_Type_II' 'Overweight_Level_I'
 'Obesity_Type_I']


In [201]:
target_order = ['Insufficient_Weight', 'Normal_Weight', 'Overweight_Level_I', 'Overweight_Level_II', 'Obesity_Type_I', 'Obesity_Type_II', 'Obesity_Type_III']


### Matriz de dispersão

In [202]:
sns.set_theme(style='ticks')
# plot = sns.pairplot(train.sample(100), plot_kws={'alpha':0.1})
# plot = sns.pairplot(train, plot_kw  s={'alpha':0.05})

## Pré-processamento


In [ ]:
# Criar o encoder com a ordem desejada
target_encoder = OrdinalEncoder(categories=[target_order], dtype=int)

train["NObeyesdad_encoded"] = target_encoder.fit_transform(
    train[["NObeyesdad"]]
).flatten()

# 3. Separar features (X) e target (y)
features = train.drop(["NObeyesdad", "NObeyesdad_encoded"], axis=1)
target = train["NObeyesdad_encoded"]


Acurácia do modelo: 0.89

Relatório de Classificação:
                     precision    recall  f1-score   support

Insufficient_Weight       0.93      0.92      0.92       524
      Normal_Weight       0.83      0.87      0.85       626
 Overweight_Level_I       0.76      0.72      0.74       484
Overweight_Level_II       0.76      0.78      0.77       514
     Obesity_Type_I       0.88      0.87      0.88       543
    Obesity_Type_II       0.97      0.98      0.97       657
   Obesity_Type_III       1.00      1.00      1.00       804

           accuracy                           0.89      4152
          macro avg       0.88      0.87      0.88      4152
       weighted avg       0.89      0.89      0.89      4152



## Treinamento


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    features, target, test_size=0.2, random_state=42
)

preprocessor = ColumnTransformer(
    transformers=[
        ('numerical', 'passthrough', X_train.select_dtypes(include=['int64', 'float64']).columns.tolist()),
        ('Gender', OneHotEncoder(drop='first', sparse_output=False), ['Gender']),
        ('CALC', OneHotEncoder(categories=[CALC_options], drop='first', sparse_output=False), ['CALC']),
        # ('CALC', OrdinalEncoder(categories=[CALC_options], dtype=int), ['CALC']),
        ('MTRANS', OneHotEncoder(drop='first', sparse_output=False), ['MTRANS'])
    ]
)

model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', RandomForestClassifier())
])

model.fit(X_train, y_train)


y_pred = model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f"\nAcurácia do modelo: {accuracy:.2f}")

print("\nRelatório de Classificação:")
print(
    classification_report(
        y_test,
        y_pred,
        target_names=target_order,
    )
)

## Predição


In [ ]:
prediction = model.predict(test)
predicted_classes = target_encoder.inverse_transform(prediction.reshape(-1, 1)).flatten()

predicted_df = pd.DataFrame({
    'id': test.id,
    'NObeyesdad': predicted_classes
})

predicted_df.to_csv('../submission/submission.csv', index=False)